# Understand embeddings with Word2Vec

### Exercise objectives:
- Convert words to vector representations thanks to embeddings
- Discover the powerful Word2Vec algorithm

<hr>

_Embeddings_ are representations of words thanks to vectors. These embeddings can be learned within a Neural Network. But it can take time to converge. Another option is to learn them as a first step. Then, use them directly to feed the word representation into an RNN. 



# The data

Keras provides many datasets, among which is the ÌMDB dataset: it is comprised of sentences that are movie reviews. Each of them is related to a score given by the review writer.

❓ **Question** ❓ Let's first load the data. You don't have to understand what is going on in the function, it does not matter here.

⚠️ **Warning** ⚠️ The `load_data` function has a `percentage_of_sentences` argument. Depending on your computer, there are chances that too many sentences will make your compute slow down, or even freeze - your RAM can overflow. For that reason, **you should start with 10% of the sentences** and see if your computer handles it. Otherwise, rerun with a lower number.  

⚠️ **DISCLAIMER** ⚠️ **No need to play _who has the biggest_ (RAM) !** The idea is to get to run your models quickly to prototype. Even in real life, it is recommended that you start with a subset of your data to loop and debug quickly. So increase the number only if you are into getting the best accuracy. 

In [130]:
###########################################
### Just run this cell to load the data ###
###########################################

import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.text import text_to_word_sequence

def load_data(percentage_of_sentences=None):
    train_data, test_data = tfds.load(name="imdb_reviews", split=["train", "test"], batch_size=-1, as_supervised=True)

    train_sentences, y_train = tfds.as_numpy(train_data)
    test_sentences, y_test = tfds.as_numpy(test_data)
    
    # Take only a given percentage of the entire data
    if percentage_of_sentences is not None:
        assert(percentage_of_sentences> 0 and percentage_of_sentences<=100)
        
        len_train = int(percentage_of_sentences/100*len(train_sentences))
        train_sentences, y_train = train_sentences[:len_train], y_train[:len_train]
  
        len_test = int(percentage_of_sentences/100*len(test_sentences))
        test_sentences, y_test = test_sentences[:len_test], y_test[:len_test]
    
    X_train = [text_to_word_sequence(_.decode("utf-8")) for _ in train_sentences]
    X_test = [text_to_word_sequence(_.decode("utf-8")) for _ in test_sentences]
    
    return X_train, y_train, X_test, y_test

X_train, y_train, X_test, y_test = load_data(percentage_of_sentences=70)

In the previous exercise, we jointly learned a representation for the words, and fed this representation to a RNN, as shown here: 

<img src="layers_embedding.png" width="400px" />

However, this increases the number of parameters to learn, which slows down and increases the difficulty of convergence!

For this reason, we will separate the steps of learning the word representation and feeding it into a RNN. As shown here: 

<img src="word2vec_representation.png" width="400px" />

We will learn the embedding with word2vec.

The drawback is indeed that the learned embeddings are not _specifically_ designed for our task. However, learning them independently of the task at hand (sentiment analysis) has some advantages: 
- it is very fast to do in general (with word2vec)
- the representation learned by word2vec is still meaningful 
- the convergence of the RNN alone will be easier and faster

So let's learn an embedding with word2vec and see how meaningful it is!


# Embedding with Word2Vec

Let's use Word2Vec to embed the words of our sentences. Word2Vec will be able to convert each word to a fixed-size vectorial representation.

For instance, we will have:
- 'dog' --> [0.1, -0.3, 0.8]
- 'cat' --> [-1.1, 2.3, 0.7]
- 'apple' --> [3.1, 0.9, -4.7]

Here, your embedding space is of size 3.

What you expect is to have representation such as words with close meanings are close in this embedding space. As in this example:

![Embedding](word_embedding.png)

❓ **Question** ❓ Let's run Word2Vec! The following code imports word2vec from [GENSIM](https://radimrehurek.com/gensim/), a great python package that makes the use of the word2vec algorithm easy, fast and accurate - which is not an easy task. The second line learns the embedding representation of the words thanks to the sentences in `X_train`. The third line stores the words and their trained embeddings in `wv`.

In [131]:
from gensim.models import Word2Vec

word2vec = Word2Vec(sentences=X_train)
wv = word2vec.wv

In [132]:
wv['dog']

array([-1.4913630e+00,  8.9403260e-01,  4.9163970e-01, -3.4331566e-01,
        2.9466817e-01,  7.6832092e-01,  6.3931793e-01,  7.7108938e-01,
        6.5140349e-01, -2.8027076e-01,  2.2428095e-01, -4.7459462e-01,
        4.8259422e-01,  3.0950472e-01, -6.8044770e-01, -5.0758666e-01,
        1.0312297e-03, -1.3765615e+00,  5.3637475e-01,  1.6336304e-01,
        1.2659920e+00, -1.0247645e+00, -7.7461261e-01, -1.1507120e+00,
       -5.9719396e-01,  2.1474150e-01, -6.0451794e-01, -8.2308225e-02,
        1.5649008e-02, -3.7676901e-01,  7.0711851e-01,  4.9012890e-01,
        3.6079466e-01,  1.7972614e-01, -3.9586801e-02,  8.1918186e-01,
       -5.0826478e-01, -4.8815000e-01, -5.9122854e-01, -3.5578120e-01,
       -3.5885540e-01, -1.8414541e-01, -2.6137197e-01, -8.4693462e-02,
       -1.1518287e-01, -1.3404260e+00,  1.3075042e+00, -2.1558918e-01,
        1.9674098e-01, -5.3597850e-01,  8.6294639e-01,  4.8558894e-01,
       -6.8404174e-01, -9.3879335e-02, -1.1654700e+00, -3.2173616e-01,
      

Let's look at the embedded representation of some words.

You can use `wv` as a dictionary.
For instance, `wv['dog']` will return a representation of `dog` in the embedding space.

❓ **Question** ❓ Try different words - especially, try non-existing words to see that they don't have any representation (which is perfectly normal as their representation was not learned). 

❓ **Question** ❓ What is the size of each word representation, and therefore, what is the size of the embedding space?

In [133]:
size = len(wv['car'])

print(f"The embedding space is of size {size}")

The embedding space is of size 100


How to know if this embedding make any sense? To do that, we will check that words with a close meaning have close representations. 

Let's use the [`Word2Vec.wv.most_similar`](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.most_similar) method that, given an input word, displays the "closest" words in the embedding space. If the embedding is well done, then words that have similar meanings will have similar representation in the embedding space.

❓ **Question** ❓ Test the `most_similar` method on different words. 

❗ **Remark** ❗ Indeed, the quality of the closeness will depend on the quality of your embedding, and thus, depend on the number of sentences that you have loaded and from which you create your embedding.

In [134]:
wv.most_similar('dog')

[('cat', 0.7729499936103821),
 ('knife', 0.6921985149383545),
 ('tree', 0.6908868551254272),
 ('fish', 0.6729645133018494),
 ('demon', 0.6719460487365723),
 ('horse', 0.663831889629364),
 ('accidentally', 0.6623042821884155),
 ('bowl', 0.6586986780166626),
 ('shirt', 0.6573867797851562),
 ('dude', 0.641177773475647)]

Similarly to `most_similar` used on words directly, we can use [`similar_by_vector`](https://radimrehurek.com/gensim/models/keyedvectors.html#gensim.models.keyedvectors.KeyedVectors.similar_by_vector) on vectors to do the same thing:

In [135]:
wv.similar_by_vector(wv['car'])

[('car', 1.0000001192092896),
 ('truck', 0.8192116022109985),
 ('room', 0.7984217405319214),
 ('gun', 0.7783563733100891),
 ('horse', 0.7781437635421753),
 ('plane', 0.767379641532898),
 ('crash', 0.7392587065696716),
 ('door', 0.7323118448257446),
 ('head', 0.7309288382530212),
 ('boat', 0.7287455797195435)]

# Arithmetic on words

Now, let's do mathematical operations on words - meaning on their vector representations!

As any word is represented as a vector, you can do basic arithmetic as:

$$W2V(good) - W2V(bad)$$

❓ **Question** ❓ Do this mathematical operation and print the result

In [108]:
wv['good'] - wv['bad']

array([-1.6598654 , -0.27266106,  0.36343405,  0.19201267, -0.1724559 ,
        0.21274471,  0.5074575 , -0.9242461 , -1.3047738 ,  0.5740302 ,
        0.2787058 ,  1.3955214 ,  1.4485329 , -1.6598843 ,  0.24725544,
        2.4285624 , -0.36306962,  0.09941307, -1.2657907 , -0.25297147,
       -0.55330944,  1.9003022 , -0.09121287,  0.7832701 , -1.2856257 ,
       -0.18477166, -0.31555125,  0.6325501 , -0.7030227 , -1.0515871 ,
       -2.1156049 ,  1.9147455 ,  0.5536448 ,  0.27735543, -3.3107677 ,
        0.9868896 , -1.2289999 ,  0.24936175, -0.42095715, -1.3034273 ,
       -0.5692983 ,  0.49445325,  1.8515042 ,  1.1266134 , -0.66426075,
        2.772113  ,  0.80453295, -0.5183238 ,  0.10235274, -0.04812169,
        0.6028429 , -1.5701523 , -0.62885237, -1.9440162 , -0.44090164,
        0.08284231, -0.45175242,  0.40387344, -0.19083905,  0.0642271 ,
        1.379005  , -0.7054977 , -0.7169875 ,  0.55854917,  1.6225259 ,
       -0.12714207,  2.0389338 ,  2.0178764 ,  1.8119586 , -1.05

Now, imagine for a second that the following equality holds true:

$$W2V(good) - W2V(bad) = W2V(nice) - W2V(stupid)$$

which is equivalent to:

$$W2V(good) - W2V(bad) + W2V(stupid) = W2V(nice)$$

❓ **Question** ❓ Let's, just for fun (as it would be bold of us to think that this equality holds true ...), do the operation $W2V(good) - W2V(bad) + W2V(stupid)$ and store it in a `res` variable (which will be a vector of size 100 that you can print).

In [109]:
res = wv['good'] - wv['bad'] + wv['stupid']
len(res)

100

We said earlier, that for any vector it is possible to see the closest vectors in the embedding space.

❓ **Question** ❓ Look at the closest vector (thanks to the `similar_by_vector` function) of `res`

In [110]:
wv.similar_by_vector(res)

[('good', 0.7085158228874207),
 ('nice', 0.6724364757537842),
 ('cool', 0.5973302721977234),
 ('great', 0.5791015028953552),
 ('decent', 0.5513383150100708),
 ('fine', 0.5478663444519043),
 ('cute', 0.5322198271751404),
 ('interesting', 0.5284685492515564),
 ('funny', 0.5215151309967041),
 ('creepy', 0.5210781693458557)]

Incredible right! You can do arithmetic operations on words!

❓ **Question** ❓ You can try on arithmetic such as 

$$W2V(Boy) - W2V(Girl) = W2V(Man) - W2V(Woman)$$

or 

$$W2V(Queen) - W2V(King) = W2V(actress) - W2V(actor)$$

❗ **Remark** ❗ You will probably see that the results are not perfect. But don't forget that you trained your model on a very small corpus.

In [111]:
res = wv['queen'] - wv['king'] + wv['actor']
wv.similar_by_vector(res)

[('actor', 0.8817663192749023),
 ('actress', 0.7942054867744446),
 ('role', 0.699042022228241),
 ('performance', 0.617006242275238),
 ('villain', 0.6078392863273621),
 ('performer', 0.588433563709259),
 ('accent', 0.5871647596359253),
 ('comedian', 0.5820879936218262),
 ('artist', 0.5673787593841553),
 ('talent', 0.5621094107627869)]

You might wonder where does this magic comes from (at quite a low price, you just run a line of code on a very small corpus and it was trained within few minutes). The magic comes from the way Word2Vec is trained. The details are quite complex, but you can remember that Word2vec, in `word2vec = Word2Vec(sentences=X_train)`, actually trains a internal neural network (that you don't see).  

In a nutshell, this internal NN predicts a word from the surroundings words in a sentences. So it splits the original sentences, then for each split it chooses some words as inputs $X$ and a word as the output $y$ which it tries to predict, using the embedding space.

And as with any neural network, Word2Vec has some hyperparameters. Let's check some out. 


# Word2Vec hyperparameters


❓ **Question** ❓ The first important hyperparameter is the `vector_size` argument. It corresponds to the size of the embedding space. Learn a new `word2vec_2` model, still trained on the `X_train`, but with a smaller or higher `vector_size`.

Verify on some words that the corresponding embedding is of your selected size.

In [112]:
word2vec_2 = Word2Vec(sentences=X_train, vector_size=50)
wv2 = word2vec_2.wv
len(wv2['movie'])

50

In [113]:
len(wv['movies'])

100

❓ **Question** ❓ Use the `Word2Vec.wv.key_to_index` attribute to display the size of the learned vocabulary. Compare it to the number of different words in `X_train`.

In [114]:
wv2.key_to_index

{'the': 0,
 'and': 1,
 'a': 2,
 'of': 3,
 'to': 4,
 'is': 5,
 'br': 6,
 'in': 7,
 'it': 8,
 'i': 9,
 'this': 10,
 'that': 11,
 'was': 12,
 'as': 13,
 'for': 14,
 'with': 15,
 'movie': 16,
 'but': 17,
 'film': 18,
 'on': 19,
 'not': 20,
 'you': 21,
 'his': 22,
 'are': 23,
 'have': 24,
 'be': 25,
 'he': 26,
 'one': 27,
 'all': 28,
 'at': 29,
 'by': 30,
 'an': 31,
 'they': 32,
 'so': 33,
 'who': 34,
 'from': 35,
 'like': 36,
 'her': 37,
 'or': 38,
 'just': 39,
 'about': 40,
 'out': 41,
 "it's": 42,
 'if': 43,
 'has': 44,
 'some': 45,
 'there': 46,
 'what': 47,
 'good': 48,
 'very': 49,
 'more': 50,
 'when': 51,
 'up': 52,
 'time': 53,
 'my': 54,
 'no': 55,
 'even': 56,
 'she': 57,
 'would': 58,
 'which': 59,
 'only': 60,
 'story': 61,
 'really': 62,
 'see': 63,
 'had': 64,
 'their': 65,
 'can': 66,
 'were': 67,
 'me': 68,
 'well': 69,
 'than': 70,
 'we': 71,
 'much': 72,
 'been': 73,
 'do': 74,
 'bad': 75,
 'get': 76,
 'will': 77,
 'also': 78,
 'people': 79,
 'other': 80,
 'first': 81,
 '

In [115]:
print('Vocabulary size', len(wv2.key_to_index))

diff_words = set([_ for words in X_train for _ in words])
print('Number of different words in the train set', len(diff_words))

Vocabulary size 25497
Number of different words in the train set 75233


In [116]:
lst = []
for words in X_train:
    for word in words:
        lst.append(word)

In [117]:
res = set(lst)

In [118]:
len(res)

75233

There is an important difference between the number of words in the train sentences and in the Word2Vec vocabulary, even though it has been trained on the train sentence set. The reasons comes from the second important hyperparameter of Word2Vec:  `min_count`. 

`min_count` is a integer that tells you how many occurrences a given word should have to be learned in the embedding space. For instance, let's say that the word "movie" appears 1000 times in the corpus and "simba" only 2 times. If `min_count=3`, the word "simba" will be skipped during the training.

The intention is to have only words that are sufficiently present in the corpus to have a robust embedded representation

❓ **Question** ❓ Learn a new `word2vec_3` model with a `min_count` higher than 5 (which is the default value) and a `word2vec_4` with a `min_count` smaller than 5, and then, compare the size of the vocabulary for all the different word2vecs that you have trained (you can choose any `vector_size` you want).

In [119]:
word2vec_3 = Word2Vec(sentences=X_train, vector_size=50, min_count=6)
wv3 = word2vec_3.wv
len(wv2['movie'])

50

In [120]:
print('Vocabulary size', len(wv3.key_to_index))

diff_words = set([_ for words in X_train for _ in words])
print('Number of different words in the train set', len(diff_words))

Vocabulary size 22987
Number of different words in the train set 75233


In [121]:
word2vec_4 = Word2Vec(sentences=X_train, vector_size=50, min_count=3)
wv4 = word2vec_4.wv
len(wv2['movie'])

50

In [122]:
print('Vocabulary size', len(wv4.key_to_index))

diff_words = set([_ for words in X_train for _ in words])
print('Number of different words in the train set', len(diff_words))

Vocabulary size 34555
Number of different words in the train set 75233


Remember that we said word2vec has an internal neural network that it optimizes based on some predictions? These predictions actually correspond to predicting a word based on surrounding words. The surroundings words are in a `window` which corresponds to the number of words taken into account. And you can train the word2vec with different `window` sizes.

❓ **Question** ❓ Train a new `word2vec_5` model with a `window` different than previously (default is 5).

In [123]:
word2vec_5 = Word2Vec(sentences=X_train, vector_size=50, min_count=3, window=20)
wv5 = word2vec_5.wv
len(wv2['movie'])

50

The arguments you have seen (`vector_size`, `min_count` and `window`) are usually the ones that you should start changing to get a better performance for your model.

But you can also look at other arguments in the [documentation](https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Text8Corpus)



# Convert our train and test set to RNN ready data

Remember that word2vec is the first step to the overall process of feeding such a representation into a RNN, as shown here:

<img src="word2vec_representation.png" width="400px" />



Now, let's work on Step 2 by converting the training and test data into their vector representation to be ready to be feed in RNNs.

❓ **Question** ❓ Now, write a function that, given a sentence, returns a matrix that corresponds to the embedding of the full sentence, which means that you have to embed each word one after the other and concatenate the result to output a 2D matrix (be sure that your output is a NumPy array)

❗ **Remark** ❗ You will probably notice that some words you are trying to convert throw errors as they are said not to belong to the dictionary:

- for the test set, this is understandable: some words were not in the train set and thus their embedded representation is unknown
- for the train set, due to `min_count` hyperparameter, not all the words have a vector representation

In any case, just skip the missing words here.

In [136]:
import numpy as np

example = ['this', 'movie', 'is', 'the', 'worst', 'action', 'movie', 'ever']
example_missing_words = ['this', 'movie', 'is', 'laaaaaaaaaame']

def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
                
    return np.array(embedded_sentence)
    
### Checks
embedded_sentence = embed_sentence(word2vec, example)
assert(type(embedded_sentence) == np.ndarray)
assert(embedded_sentence.shape == (8, 100))

embedded_sentence_missing_words = embed_sentence(word2vec, example_missing_words)  
assert(type(embedded_sentence_missing_words) == np.ndarray)
assert(embedded_sentence_missing_words.shape == (3, 100))

In [137]:
embed_sentence(word2vec, example)

array([[ 1.192783  , -0.28507078, -0.7674951 , -1.1292183 ,  0.80440176,
         0.5661437 ,  0.32600012, -0.43060878, -0.8692231 ,  1.9584227 ,
         0.62567925, -0.54841614,  1.474724  , -0.36395544, -0.87344044,
         0.182759  , -0.2154458 ,  1.300823  ,  2.0341105 ,  1.0282305 ,
         0.9756102 ,  0.5863852 ,  0.85646933, -0.21970701,  0.9204446 ,
        -0.39124578,  0.44561476, -1.6884341 ,  1.0476317 , -2.2650971 ,
         0.9297659 , -1.3270738 ,  1.0791148 ,  0.6630168 ,  0.24287547,
        -2.0041094 , -1.7934724 , -2.1063154 ,  0.98667437, -2.2306178 ,
        -0.3796588 , -1.9244324 , -2.853235  , -0.5893364 , -2.05958   ,
         1.4219428 , -1.3539181 , -1.5276321 , -1.7691156 , -1.5968212 ,
        -2.3911343 , -1.587346  , -1.9070529 , -1.1233429 ,  0.9309436 ,
        -0.14852998, -0.72603315, -0.69915146, -0.81811506, -2.5763845 ,
        -0.7320825 , -2.5250025 , -1.0410788 , -2.1569653 , -2.1186404 ,
        -1.5052265 , -1.6551435 , -1.4500695 , -1.7

❓ **Question** ❓ Write a function that, given a list of sentences (each sentence being a list of words/strings), returns a list of embedded sentences (each sentence is a matrix). Apply this function to the train and test sentences

Hint: Use the previous function `embed_sentence`

In [138]:
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed
        
X_train = embedding(word2vec, X_train)
X_test = embedding(word2vec, X_test)

In [139]:
X_train[0]

array([[ 1.192783  , -0.28507078, -0.7674951 , ...,  1.2996094 ,
         1.3883663 , -2.0431354 ],
       [ 1.9710703 , -3.624081  ,  0.9539811 , ..., -1.2825993 ,
         3.626972  ,  1.1842213 ],
       [-1.4513208 , -0.4360151 ,  1.0131621 , ...,  1.2721511 ,
        -0.9402817 , -2.602903  ],
       ...,
       [ 0.08794045, -2.3368826 ,  0.6784791 , ..., -0.8496931 ,
        -0.13602793,  3.1434748 ],
       [ 0.7677526 ,  1.3683906 ,  0.2987402 , ..., -0.7672979 ,
         1.3430125 ,  0.34882003],
       [ 0.9785652 , -1.02191   , -1.6895281 , ..., -0.42424074,
         0.9731899 , -0.55704427]], dtype=float32)

❓ **Question** ❓ In order to have ready-to-use data, do not forget to pad your sequences so you have tensors that can be divided into batches (of `batch_size`) during the optimization. Store the padded values in `X_train_pad` and `X_test_pad`. Do not forget the important arguments of the padding ;)

In [140]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X_train_pad = pad_sequences(X_train, dtype='float32', padding='post')
X_test_pad = pad_sequences(X_test, dtype='float32', padding='post')

assert(len(X_train_pad.shape) == 3)
assert(len(X_test_pad.shape) == 3)
assert(X_train_pad.shape[2] == 100)
assert(X_test_pad.shape[2] == 100)